# Pre-processing of the outputevents dataset

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

file_path="~/Documents/Data/Full_MIMIC/"

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 300)

In [2]:
adm=pd.read_csv(file_path+"Admissions_processed.csv")

We now consider the outputevents dataset. We select only the patients with the same criteria as above.

In [3]:
outputs=pd.read_csv(file_path+"OUTPUTEVENTS.csv")

In [4]:
#Some checks
assert(len(outputs.loc[outputs["ISERROR"].notnull()].index)==0) #No entry with iserror==TRUE

#Restrict the dataset to the previously selected admission ids only.
adm_ids=list(adm["HADM_ID"])
outputs=outputs.loc[outputs["HADM_ID"].isin(adm_ids)]

print("Number of patients remaining in the database: ")
print(outputs["SUBJECT_ID"].nunique())

Number of patients remaining in the database: 
23330


We load the D_ITEMS dataframe which contains the name of the ITEMID. And we merge both tables together.

In [5]:
#item_id 
item_id=pd.read_csv(file_path+"D_ITEMS.csv")
item_id_1=item_id[["ITEMID","LABEL"]]
item_id_1.head()

#We merge the name of the item administrated.
outputs_2=pd.merge(outputs,item_id_1,on="ITEMID")
outputs_2.head()
print("Number of patients remaining in the database: ")
print(outputs_2["SUBJECT_ID"].nunique())

Number of patients remaining in the database: 
23330


We compute the number of patients that have the specific outputs labels and we select only the features that are the most present over the whole data set. For this, we rank the features by number of patients and select the n_best.

In [6]:
n_best=15
#For each item, evaluate the number of patients who have been given this item.
pat_for_item=outputs_2.groupby("LABEL")["SUBJECT_ID"].nunique()
#Order by occurence and take the 20 best (the ones with the most patients)
frequent_labels=pat_for_item.sort_values(ascending=False)[:n_best]

#Select only the time series with high occurence.
outputs_3=outputs_2.loc[outputs_2["LABEL"].isin(list(frequent_labels.index))].copy()

print("Number of patients remaining in the database: ")
print(outputs_3["SUBJECT_ID"].nunique())
print("Number of datapoints remaining in the database: ")
print(len(outputs_3.index))

print(frequent_labels)

Number of patients remaining in the database: 
23201
Number of datapoints remaining in the database: 
1758750
LABEL
Urine Out Foley                              12073
Foley                                         9150
Stool Out Stool                               4800
Chest Tubes CTICU CT 1                        3483
Gastric Oral Gastric                          3304
OR Urine                                      3291
Urine Out Void                                3228
Void                                          3220
OR Out PACU Urine                             2904
Chest Tube #1                                 2527
Pre-Admission Output Pre-Admission Output     2069
Pre-Admission                                 1854
Oral Gastric                                  1781
OR Out OR Urine                               1723
OR Out EBL                                    1720
Name: SUBJECT_ID, dtype: int64


#### Eventually, we select the same labels of the paper

In [25]:
outputs_label_list=['Gastric Gastric Tube','Stool Out Stool','Urine Out Incontinent','Ultrafiltrate Ultrafiltrate','Foley', 'Void','Condom Cath','Fecal Bag','Ostomy (output)','Chest Tube #1','Chest Tube #2','Jackson Pratt #1','OR EBL','Pre-Admission','TF Residual']
outputs_bis=outputs_2.loc[outputs_2["LABEL"].isin(outputs_label_list)].copy()

print("Number of patients remaining in the database: ")
print(outputs_bis["SUBJECT_ID"].nunique())
print("Number of datapoints remaining in the database: ")
print(len(outputs_bis.index))

outputs_3=outputs_bis.copy()

Number of patients remaining in the database: 
15087
Number of datapoints remaining in the database: 
750817


# Cleaning of the output data

### Units Cleaning

#### 1) Amounts

In [26]:
#Verification that all input labels have the same amounts units.
outputs_3.groupby("LABEL")["VALUEUOM"].value_counts() #OK

LABEL                        VALUEUOM
Chest Tube #1                mL           58749
Chest Tube #2                mL            7534
Condom Cath                  mL            2392
Fecal Bag                    mL            1263
Foley                        mL          577219
Gastric Gastric Tube         ml            1616
Jackson Pratt #1             mL            8168
OR EBL                       mL            1940
Ostomy (output)              mL            1562
Pre-Admission                mL            2530
Stool Out Stool              ml           29354
TF Residual                  mL           11871
Ultrafiltrate Ultrafiltrate  ml           18463
Urine Out Incontinent        ml             861
Void                         mL           26802
Name: VALUEUOM, dtype: int64

### Check for outliers

#### 1) In amounts

In [27]:
outputs_3.groupby("LABEL")["VALUE"].describe()

,count,mean,std,min,25%,50%,75%,max
LABEL,,,,,,,,
Chest Tube #1,58749.0,37.339165,59.338525,0.0,10.0,30.0,50.0,2500.0
Chest Tube #2,7534.0,31.527741,63.450023,0.0,0.0,20.0,40.0,1900.0
Condom Cath,2392.0,244.961538,223.463992,0.0,100.0,200.0,340.0,2350.0
Fecal Bag,1263.0,447.680918,341.054355,0.0,200.0,375.0,700.0,2500.0
Foley,577219.0,127.286087,6026.738198,0.0,45.0,80.0,150.0,4555555.0
Gastric Gastric Tube,1676.0,118.635442,156.237053,0.0,15.0,60.0,175.0,1600.0
Jackson Pratt #1,8168.0,72.272857,100.280253,0.0,20.0,40.0,80.0,1325.0
OR EBL,1940.0,694.788660,1499.586855,0.0,100.0,250.0,600.0,20000.0
Ostomy (output),1562.0,204.490397,169.469766,0.0,100.0,175.0,300.0,2000.0


In [28]:
#Remove all entries whose rate is more than 4 std away from the mean.
out_desc=outputs_3.groupby("LABEL")["VALUE"].describe()
name_list=list(out_desc.loc[out_desc["count"]!=0].index)
for label in name_list:
    outputs_3=outputs_3.drop(outputs_3.loc[(outputs_3["LABEL"]==label)&(outputs_3["VALUE"]>(out_desc.loc[label,"mean"]+4*out_desc.loc[label,"std"]))].index).copy()

print("Number of patients remaining in the database: ")
print(outputs_3["SUBJECT_ID"].nunique())
print("Number of datapoints remaining in the database: ")
print(len(outputs_3.index))

Number of patients remaining in the database: 
15057
Number of datapoints remaining in the database: 
749738


In [29]:
#Clean Foley, remove too large values
outputs_3=outputs_3.drop(outputs_3.loc[(outputs_3["LABEL"]=="Foley") & (outputs_3["VALUE"]>5500)].index).copy()
#Clean Expected Blood Loss, remove too large values
outputs_3=outputs_3.drop(outputs_3.loc[(outputs_3["LABEL"]=="OR EBL") & (outputs_3["VALUE"]>5000)].index).copy()
#Clean Out Expected Blood Loss, remove too large values
outputs_3=outputs_3.drop(outputs_3.loc[(outputs_3["LABEL"]=="OR Out EBL") & (outputs_3["VALUE"]>5000)].index).copy()
#Clean OR Urine, remove too large values
outputs_3=outputs_3.drop(outputs_3.loc[(outputs_3["LABEL"]=="OR Urine") & (outputs_3["VALUE"]>5000)].index).copy()
#Clean Pre-Admission, remove too large and negative values
outputs_3=outputs_3.drop(outputs_3.loc[(outputs_3["LABEL"]=="Pre-Admission") & (outputs_3["VALUE"]<0)].index).copy()
outputs_3=outputs_3.drop(outputs_3.loc[(outputs_3["LABEL"]=="Pre-Admission") & (outputs_3["VALUE"]>5000)].index).copy()
#Clean Pre-Admission output, remove too large values
outputs_3=outputs_3.drop(outputs_3.loc[(outputs_3["LABEL"]=="Pre-Admission Output Pre-Admission Output") & (outputs_3["VALUE"]>5000)].index).copy()
#Clean Urine Out Foley output, remove too large values
outputs_3=outputs_3.drop(outputs_3.loc[(outputs_3["LABEL"]=="Urine Out Foley") & (outputs_3["VALUE"]>5000)].index).copy()
#Clean Void, remove negative values
outputs_3=outputs_3.drop(outputs_3.loc[(outputs_3["LABEL"]=="Void") & (outputs_3["VALUE"]<0)].index).copy()

outputs_3.dropna(subset=["VALUE"],inplace=True)

print("Number of patients remaining in the database: ")
print(outputs_3["SUBJECT_ID"].nunique())
print("Number of datapoints remaining in the database: ")
print(len(outputs_3.index))

Number of patients remaining in the database: 
12303
Number of datapoints remaining in the database: 
726834


As data is already in timestamp format, we don't neeed to consider rates

In [30]:
outputs_3.to_csv(file_path+"OUTPUTS_processed.csv")